The scripts should be run in order.
<br> Currently you are on 
<br>&ensp;->&nbsp;resume.ipynb
<br>&emsp;&emsp;job_selection.ipynb or job_selection2.ipynb
<br>&emsp;&emsp;job_vectorise.ipynb

Task3 - reassesed the requirements, now using dataframes more

In [1]:

import pandas as pd
import os, glob
from glob import glob
from bs4 import BeautifulSoup
import logging, importlib, sys, tqdm
import spacy
import pickle
from _pckle import save_pickle_object, load_pickle_object
from _logging import set_logging
from _graph import histplot_count, histplot_range_count
from _utility import gl

set_logging(logging)

In [2]:
def read_html_file(fname):
    with open (fname, 'r', encoding="utf8") as f:
        doc = BeautifulSoup(f.read(), 'html.parser')
        return doc

In [3]:
def tokenise_words(text, nlp):
    word_tokens = nlp(text)
    # converts the words in word_tokens to lower case and then checks whether
    # they are present in stop_words or not
    tokens = [w for w in word_tokens if not w.is_stop and not w.is_punct and not w.is_space]

    return tokens

In [4]:
def get_body(tag_body):
    body = " ".join(tag_body.strings)
    return body


In [5]:
def get_bullet_points(tag_body):
    get_text = lambda x: " " + x.get_text(separator=" ")
    
    tag_bullet_point = "li"
    bullet_points = ""
    for item in tag_body.find_all(tag_bullet_point):  
        bullet_points += get_text(item)
    return bullet_points

Read the resume

In [6]:
def get_resume(file, nlp):
    file_path = os.path.join("Files\\UserInput", file)
    f = open(file_path, "r")
    resume_text = f.read()
    nlp_resume_text = nlp(resume_text)
    return resume_text, nlp_resume_text


Get the cosine similarity score between the resume and the job description

In [7]:
def get_cosine_similarity_score(body, nlp_resume_text, nlp):
    nlp_job = nlp(body)
    score = nlp_resume_text.similarity(nlp_job)
    return score

Create output folders

In [8]:
output_folders = ["Files\\Output", "Images", "Images_tfidf"]
for folder in output_folders:
    if os.path.exists(folder):
        continue
    os.makedirs(folder) 

Load all the job descriptions from the HTML files into a dataframe

In [9]:
nlp = spacy.load('en_core_web_md')
resume_text, nlp_resume_text = get_resume('resume.txt', nlp)
files = glob(os.path.join("Files\\Input", '*.html'))
col_names =  [gl.id, gl.title, gl.body, gl.bullet_points, gl.score]
df_jobs = pd.DataFrame(columns = col_names)

for i, fname in enumerate(files):
    if i % 100 == 0:
        logging.info(f"{i}th file")
    message = f"{i+1}. file : {fname}"
    logging.info(message)
    doc = read_html_file(fname)
    title = doc.title.string
    body = get_body(doc.body)
    bullet_points = get_bullet_points(doc.body)
    score = get_cosine_similarity_score(body, nlp_resume_text, nlp)
    df_jobs.loc[len(df_jobs)] = [i, title, body, bullet_points, score]


save_pickle_object(df_jobs, gl.pkl_df_jobs_file)

2023-01-02 14:41:39,672 | INFO : 0th file
2023-01-02 14:41:39,673 | INFO : 1. file : Files\Input\001b92395ed0fb62.html
2023-01-02 14:41:39,739 | INFO : 2. file : Files\Input\00321a48d04fe754.html
2023-01-02 14:41:39,829 | INFO : 3. file : Files\Input\0079c11b2611349f.html
2023-01-02 14:41:39,904 | INFO : 4. file : Files\Input\007d9d7b5c09d820.html
2023-01-02 14:41:39,977 | INFO : 5. file : Files\Input\00bf37ae19c7dfd7.html
2023-01-02 14:41:40,110 | INFO : 6. file : Files\Input\0125eabc844281c9.html
2023-01-02 14:41:40,190 | INFO : 7. file : Files\Input\014ae4dbded805d2.html
2023-01-02 14:41:40,266 | INFO : 8. file : Files\Input\014ea972a4aa9812.html
2023-01-02 14:41:40,358 | INFO : 9. file : Files\Input\0179ea131f141400.html
2023-01-02 14:41:40,485 | INFO : 10. file : Files\Input\018866568cd5a0b0.html
2023-01-02 14:41:40,671 | INFO : 11. file : Files\Input\01aa3fd02a66fc1d.html
2023-01-02 14:41:40,754 | INFO : 12. file : Files\Input\01bd31cf3814dee3.html
2023-01-02 14:41:40,828 | INFO 

Having obtained the dataframe for the job descriptions and stored the data in a pickle file which takes a couple of minutes to run, proceed to the job_selection.ipynb script to determine which job descriptions to use.